<font size="5"><b>Option Pricing Models in Python</b></font>

This project implements two **European Option Pricing Models** using Object-Oriented Programming (OOP):
1. **Black-Scholes Model**
2. **Binomial Tree Model**

In [3]:
import numpy as np
from scipy.stats import norm


class OptionPricer:
    def __init__(self, S, K, T, r, sigma, option_type):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        if option_type.lower() not in ["call", "put"]:
            raise ValueError("option_type must be either 'call' or 'put'")
        self.option_type = option_type.lower()

    def get_price(self):
        if self.option_type == "call":
            return self.call_price()
        elif self.option_type == "put":
            return self.put_price()


In [4]:
class Black_Scholes(OptionPricer):
    def get_params(self):
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return d1, d2
        
    def call_price(self):
        d1, d2 = self.get_params()
        call_price = self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        return call_price

    def put_price(self):
        d1, d2 = self.get_params()
        put_price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)
        return put_price


option = Black_Scholes(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
print("European Call Option Price:", round(option.call_price(), 2))
print("European Put Option Price:", round(option.put_price(), 2)) 


European Call Option Price: 10.45
European Put Option Price: 5.57


In [5]:
class Binomial(OptionPricer):
    def __init__(self, S, K, T, r, sigma, n, option_type):
        super().__init__(S, K, T, r, sigma, option_type)
        self.n = n

    def get_params(self):
        deltaT = self.T / self.n
        u = np.exp(self.sigma * np.sqrt(deltaT))
        d = 1 / u
        p = (np.exp(self.r * deltaT) - d) / (u - d)
        q = 1 - p
        return deltaT, u, d, p, q

    def call_price(self):
        deltaT, u, d, p, q = self.get_params()
        prices = np.array([self.S * u**i * d**(self.n - i) for i in range(self.n + 1)])
        V = np.maximum(0, prices - self.K)
        for i in range(self.n - 1, -1, -1):
            V = np.exp(-self.r * deltaT) * (p * V[1:] + q * V[:-1]) 
        return V[0]

    def put_price(self):
        deltaT, u, d, p, q = self.get_params()
        prices = np.array([self.S * u**i * d**(self.n - i) for i in range(self.n + 1)])
        V = np.maximum(0, self.K - prices)
        for i in range(self.n - 1, -1, -1):
            V = np.exp(-self.r * deltaT) * (p * V[1:] + q * V[:-1]) 
        return V[0]
       
option = Binomial(S=100, K=100, T=1, r=0.05, sigma=0.2, n=100, option_type="call")
print("European Call Option Price:", round(option.call_price(), 2))
print("European Put Option Price:", round(option.put_price(), 2)) 


European Call Option Price: 10.43
European Put Option Price: 5.55
